In [1]:
import pandas as ps
import numpy as np
import tensorflow as tf
import os
for direname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(direname, filename))

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

/kaggle/input/plant-pathology-2020-fgvc7/sample_submission.csv
/kaggle/input/plant-pathology-2020-fgvc7/train.csv
/kaggle/input/plant-pathology-2020-fgvc7/test.csv
/kaggle/input/plant-pathology-2020-fgvc7/images/Test_1743.jpg
/kaggle/input/plant-pathology-2020-fgvc7/images/Test_262.jpg
/kaggle/input/plant-pathology-2020-fgvc7/images/Train_1524.jpg
/kaggle/input/plant-pathology-2020-fgvc7/images/Train_1336.jpg
/kaggle/input/plant-pathology-2020-fgvc7/images/Train_1461.jpg
/kaggle/input/plant-pathology-2020-fgvc7/images/Train_1797.jpg
/kaggle/input/plant-pathology-2020-fgvc7/images/Test_1499.jpg
/kaggle/input/plant-pathology-2020-fgvc7/images/Train_1170.jpg
/kaggle/input/plant-pathology-2020-fgvc7/images/Test_1259.jpg
/kaggle/input/plant-pathology-2020-fgvc7/images/Test_422.jpg
/kaggle/input/plant-pathology-2020-fgvc7/images/Test_636.jpg
/kaggle/input/plant-pathology-2020-fgvc7/images/Test_1395.jpg
/kaggle/input/plant-pathology-2020-fgvc7/images/Test_1800.jpg
/kaggle/input/plant-patholog

In [2]:
!pip install segment_anything

In [3]:
!python classify_images.py


python: can't open file '/kaggle/working/classify_images.py': [Errno 2] No such file or directory


In [4]:
import torch
import torchvision
import sys
import matplotlib.pyplot as plt
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

# Load the Segment Anything model.
sam_checkpoint = "/kaggle/working/sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = "cuda"



In [5]:
import sys
!{sys.executable} -m pip install opencv-python matplotlib
!{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'
!mkdir images
!get -P images https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/dog.jpg
!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-y4jznm9b
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-y4jznm9b
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... - \ done
/bin/bash: line 1: get: command not found
--2023-06-25 22:00:48--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.160.249.12, 18.160.249.45, 18.160.249.78, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.160.249.12|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2564550879 (2.4G) [binary/octet-stream]
Saving to: ‘sam_vit_h_4b8939.pth’

sam_vit_h_4b8939.pt 100%[===================>]   2.39G   251MB/s    in 10s     

2023-06-25 22:00:59 (239 MB/s) - ‘sam_vit_

In [6]:
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)


Sam(
  (image_encoder): ImageEncoderViT(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 1280, kernel_size=(16, 16), stride=(16, 16))
    )
    (blocks): ModuleList(
      (0-31): 32 x Block(
        (norm1): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=1280, out_features=3840, bias=True)
          (proj): Linear(in_features=1280, out_features=1280, bias=True)
        )
        (norm2): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (lin1): Linear(in_features=1280, out_features=5120, bias=True)
          (lin2): Linear(in_features=5120, out_features=1280, bias=True)
          (act): GELU(approximate='none')
        )
      )
    )
    (neck): Sequential(
      (0): Conv2d(1280, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): LayerNorm2d()
      (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (3): LayerNorm2d

In [7]:
# Path to the CSV file with image names and their corresponding classes
csv_file = '/kaggle/input/plant-pathology-2020-fgvc7/train.csv'  # Replace with your CSV file

In [8]:
# Load the CSV file
import pandas as pd
train_data = pd.read_csv(csv_file)

In [9]:
train_data.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Train_0,0,0,0,1
1,Train_1,0,1,0,0
2,Train_2,1,0,0,0
3,Train_3,0,0,1,0
4,Train_4,1,0,0,0


In [10]:
# category - for each image_id
def convert_dummies_to_category(df):
    data = df.copy()
    s2 = data[['healthy','multiple_diseases','rust','scab']].idxmax(axis=1)
    data = pd.concat([data,s2],axis=1)
    data.rename({0:'category'},axis=1,inplace=True)
    return data

# apply above function to find the category of the image
train_df = convert_dummies_to_category(train_data)
train_df.head()

,image_id,healthy,multiple_diseases,rust,scab,category
0,Train_0,0,0,0,1,scab
1,Train_1,0,1,0,0,multiple_diseases
2,Train_2,1,0,0,0,healthy
3,Train_3,0,0,1,0,rust
4,Train_4,1,0,0,0,healthy


In [11]:
import os
import shutil
import pandas as pd

# Path to the directory containing the images
input_directory = '/kaggle/input/plant-pathology-2020-fgvc7'  # Replace with your input directory

# Path to the directory for organizing images into class folders
output_directory = '/kaggle/working/images'  # Replace with your desired output directory

# Path to the CSV file with image names and their corresponding classes
csv_file = os.path.join(input_directory, 'train.csv')  # Replace with your CSV file

# Load the CSV file
data = pd.read_csv(csv_file)

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Iterate over the rows of the CSV file
for index, row in train_df.iterrows():
    image_name = row['image_id']
    image_class = row['category']

    # Path to the source image file
    source_path = os.path.join(input_directory, 'images', f'{image_name}.jpg')

    # Create the class folder if it doesn't exist
    class_folder = os.path.join(output_directory, image_class)
    os.makedirs(class_folder, exist_ok=True)

    # Path to the destination image file
    destination_path = os.path.join(class_folder, f'{image_name}.jpg')

    try:
        # Copy the image file to the class folder
        shutil.copy(source_path, destination_path)
        print(f"Copied {image_name}.jpg to {class_folder}")
    except FileNotFoundError:
        print(f"Error: {image_name}.jpg not found in {input_directory}")

print("Images have been organized into class folders.")


Copied Train_0.jpg to /kaggle/working/images/scab
Copied Train_1.jpg to /kaggle/working/images/multiple_diseases
Copied Train_2.jpg to /kaggle/working/images/healthy
Copied Train_3.jpg to /kaggle/working/images/rust
Copied Train_4.jpg to /kaggle/working/images/healthy
Copied Train_5.jpg to /kaggle/working/images/healthy
Copied Train_6.jpg to /kaggle/working/images/multiple_diseases
Copied Train_7.jpg to /kaggle/working/images/scab
Copied Train_8.jpg to /kaggle/working/images/scab
Copied Train_9.jpg to /kaggle/working/images/healthy
Copied Train_10.jpg to /kaggle/working/images/rust
Copied Train_11.jpg to /kaggle/working/images/scab
Copied Train_12.jpg to /kaggle/working/images/scab
Copied Train_13.jpg to /kaggle/working/images/healthy
Copied Train_14.jpg to /kaggle/working/images/rust
Copied Train_15.jpg to /kaggle/working/images/rust
Copied Train_16.jpg to /kaggle/working/images/scab
Copied Train_17.jpg to /kaggle/working/images/scab
Copied Train_18.jpg to /kaggle/working/images/healt

In [12]:
!nvidia-smi


Sun Jun 25 22:01:33 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    27W /  70W |   3378MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla T4            Off  | 00000000:00:05.0 Off |                    0 |
| N/A   

In [13]:
!torch.cuda.set_max_split_size_mb(128)


/bin/bash: -c: line 1: syntax error near unexpected token `128'
/bin/bash: -c: line 1: `torch.cuda.set_max_split_size_mb(128)'


In [14]:
import torch
import torchvision
import sys
import matplotlib.pyplot as plt
import segment_anything as sa
import torchvision.models as models
import torchvision.models.densenet as densenet

# Load the Segment Anything model.
sam_checkpoint = "/kaggle/working/sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = "cuda"

sam = sa.sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

# Load the image dataset.
dataset = torchvision.datasets.ImageFolder(
    "/kaggle/working/images",
    transform=torchvision.transforms.ToTensor(),
)

# Segment the images.
for image, target in dataset:
    image = image.to(device=device)
    def forward(self, image, multimask_output=False):
     

        features = self.encoder(image)
        mask = self.decoder(features)

        if multimask_output:
            return mask.unsqueeze(1)
        else:
            return mask

    #mask = sam(image, multimask_output).argmax(dim=1).squeeze()

    # Plot the image and the mask.
    #plt.subplot(1, 2, 1)
    #plt.imshow(image.cpu().numpy())
    #plt.title("Image")

    #plt.subplot(1, 2, 2)
    #plt.imshow(mask.cpu().numpy(), cmap="gray")
    #plt.title("Mask")

    #plt.show()

# Train the classifier.

classifier = classifier = torchvision.models.resnet18(pretrained=True)
classifier.fc = torch.nn.Linear(512, 10)
classifier.to(device=device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(classifier.parameters(), lr=0.001)

for epoch in range(15):
    for image, target in dataset:
        #image = image.to(device=device)
        target = torch.as_tensor(target)
        target = target.to(device=device)
        image = torch.randn(3, 224, 224)

        # Add a batch dimension to the tensor.
        image = image.unsqueeze(0)
        image = image.to(device="cuda")

        output = classifier(image)
        target = target.unsqueeze(0)

        loss = criterion(output, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# Evaluate the classifier.
correct = 0
total = 0

for image, target in dataset:
    image = image.to(device=device)
    target = torch.as_tensor(target)
    target = target.to(device=device)
    
    image = image.unsqueeze(0)

    output = classifier(image)

    _, predicted = output.max(1)

    total += 1
    if predicted == target:
        correct += 1

accuracy = correct / total

print("Accuracy:", accuracy)


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 191MB/s]


Accuracy: 0.32509610104338277


In [15]:
import torch
import torchvision
import sys
import matplotlib.pyplot as plt
import segment_anything as sa
import torchvision.models as models
import torchvision.models.densenet as densenet


# Load the Segment Anything model.
sam_checkpoint = "/kaggle/working/sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = "cuda"

sam = sa.sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

# Load the image dataset.
dataset = torchvision.datasets.ImageFolder(
    "/kaggle/working/images",
    transform=torchvision.transforms.Compose(
        [
            torchvision.transforms.ToTensor(),
            torchvision.transforms.RandomHorizontalFlip(),
            torchvision.transforms.RandomVerticalFlip(),
        ]
    ),
)

# Segment the images.
for image, target in dataset:
    image = image.to(device=device)
    def forward(self, image, multimask_output=False):
     

        features = self.encoder(image)
        mask = self.decoder(features)

        if multimask_output:
            return mask.unsqueeze(1)
        else:
            return mask

    #mask = sam(image).argmax(dim=1).squeeze()


    # Plot the image and the mask.
    #plt.subplot(1, 2, 1)
    #plt.imshow(image.cpu().numpy())
    #plt.title("Image")

    #plt.subplot(1, 2, 2)
    #plt.imshow(mask.cpu().numpy(), cmap="gray")
    #plt.title("Mask")

    #plt.show()

# Train the classifier.
classifier = torchvision.models.resnet18(pretrained=True)
classifier.fc = torch.nn.Linear(512, 10)
classifier.to(device=device)

criterion = torch.nn.CrossEntropyLoss(reduction="sum")
optimizer = torch.optim.SGD(classifier.parameters(), lr=0.0001, weight_decay=0.0001)

for epoch in range(20):
    for image, target in dataset:
        image = image.to(device=device)
        target = torch.as_tensor(target)
        target = target.to(device=device)

        # Add a batch dimension to the tensor.
        image = image.unsqueeze(0)
        image = image.to(device="cuda")

        output = classifier(image)
        target = target.unsqueeze(0)

        loss = criterion(output, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# Evaluate the classifier.
correct = 0
total = 0

for image, target in dataset:
    image = image.to(device=device)
    target = torch.as_tensor(target)
    target = target.to(device=device)

    image = image.unsqueeze(0)
    output = classifier(image)

    _, predicted = output.max(1)

    total += 1
    if predicted == target:
        correct += 1

accuracy = correct / total

print("Accuracy:", accuracy)


Accuracy: 0.32509610104338277
